In [1]:
setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

library(dplyr)
library(data.table)

source("/mnt/lareaulab/reliscu/code/FindModules/FindModules.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor



Attaching package: ‘flashClust’


The following object is masked from ‘package:fastcluster’:

    hclust


The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘svMisc’


The following object is masked from ‘package:utils’:

    ?


Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:



Allowing multi-threading with up to 40 threads.


Here I run FM to (hopefully) find modules representing each of the cell types present in the single-cell data

In [ ]:
counts <- fread("data/SyntheticDatasets/SyntheticDataset1_20pcntCells_10pcntVar_200samples_11-31-12.csv", data.table=FALSE)

In [ ]:
# For duplicate genes, choose row with highest mean expression

mean_expr <- data.frame(
    Index=1:nrow(counts), 
    Gene=counts[,1], 
    Expr=rowMeans(counts[,-1])
)

mean_expr <- mean_expr %>%
    group_by(Gene) %>%
    slice_max(Expr)

print(dim(mean_expr))

counts <- counts[mean_expr$Index,]

In [ ]:
# Subset to genes in the top 50th percentile of mean expression

prob <- .1
mean_expr <- rowMeans(counts[,-1])
print(sum(mean_expr >= quantile(mean_expr, prob)))

ERROR: Error: object 'counts' not found


In [ ]:
counts_filtered <- counts[mean_expr >= quantile(mean_expr, .5),]

In [ ]:
# Normalize counts by sample

total_expr <- colSums(counts_filtered[,-1])
expr <- sweep(counts_filtered[,-1], MARGIN=2, FUN="/", STATS=total_expr) * 1e4
expr <- log2(expr + 1)
expr <- data.frame(Gene=counts_filtered[,1], expr)

In [ ]:
FindModules(
  projectname="tasic_2018_ALM_STAR_20pcntCells_10pcntVar_200samples_log2_pseudobulk",
  expr=expr,
  geneinfo=1,
  sampleindex=2:ncol(expr),
  samplegroups=NULL,
  subset=NULL,
  simMat=NULL,
  saveSimMat=FALSE,
  simType="Bicor",
  beta=1,
  overlapType="None",
  TOtype="signed",
  TOdenom="min",
  MIestimator="mi.mm",
  MIdisc="equalfreq",
  signumType="rel",
  iterate=TRUE,
  signumvec=c(.999999,.99999,.9999,.999,.99,.98,.97),
  minsizevec=c(3,4,5,6,8,10),
  signum=NULL,
  minSize=NULL,
  merge.by="ME",
  merge.param=0.9,
  export.merge.comp=T,
  ZNCcut=2,
  calcSW=FALSE,
  loadTree=FALSE,
  writeKME=TRUE,
  calcBigModStat=FALSE,
  writeModSnap=TRUE
)